In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

## 依照指示取出模型特定層的資訊

In [2]:
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16* 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()


### 取出 self.pool層兩次的輸出，包含：
* x = self.pool(F.relu(self.conv1(x)))
* x = self.pool(F.relu(self.conv2(x)))

In [8]:
model.modules

<bound method Module.modules of TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)>

In [13]:
model.conv1

Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))

In [15]:
model.pool

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

In [16]:
outputs= []
def layer1_hook(module, input_, output):
    # "自行填入"
    outputs.append(output)

# model."自行填入".register_forward_hook("自行填入")
model.pool.register_forward_hook(layer1_hook)

In [17]:
input_ = torch.randn(1, 3, 32, 32)
output = model(input_)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [18]:
outputs

[tensor([[[[0.3194, 0.6774, 0.5921,  ..., 0.2432, 0.8002, 1.1469],
           [0.4856, 0.6111, 0.2054,  ..., 0.6191, 1.1787, 0.2914],
           [0.3558, 0.0775, 0.8348,  ..., 0.7249, 0.9137, 0.1753],
           ...,
           [0.0000, 0.6320, 0.4062,  ..., 0.7708, 0.0578, 0.1531],
           [0.2052, 0.0967, 0.3755,  ..., 0.4591, 0.6471, 0.4888],
           [0.3707, 0.7332, 0.2759,  ..., 0.3470, 0.1903, 0.6998]],
 
          [[0.5715, 0.6722, 0.4968,  ..., 0.7077, 1.1991, 0.9695],
           [0.5224, 0.6837, 0.3139,  ..., 0.3582, 0.3796, 0.5480],
           [0.5289, 0.7877, 0.5397,  ..., 0.4103, 0.4002, 0.3425],
           ...,
           [0.5790, 0.4338, 0.5615,  ..., 0.9411, 0.6275, 1.2211],
           [0.3543, 0.1732, 0.7864,  ..., 0.1673, 0.8293, 0.2989],
           [0.6196, 0.1232, 0.0000,  ..., 0.5943, 1.5508, 0.0816]],
 
          [[0.8624, 1.3004, 1.0335,  ..., 0.7217, 0.5650, 0.9807],
           [0.5068, 0.3609, 0.0000,  ..., 0.8510, 0.7099, 0.6308],
           [0.2830, 1.10

In [19]:
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])


## 加入自定義 initialization fuction

#### 對所有Conv2D層使用自定義initialization function
* weight : nn.init.kaiming_normal_
* bias : 全部輸入1

In [22]:
from torch.nn import init

def weights_init(m):
    #classname = m.__class__.__name__
    # if isinstance(m, "自行填入"):
    if isinstance(m, nn.Conv2d):
        #torch.nn.init.xavier_uniform_(m.weight)
        nn.init.kaiming_normal_(m.weight.data,
                                    a=0,
                                    mode='fan_out',
                                    nonlinearity='relu')
        if m.bias is not None:
            # "自行填入"
            torch.nn.init.ones_(m.bias)
            
# model.apply("自行填入")
model.apply(fn=weights_init)

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

#### 查看 conv層的bias是否皆為1

In [23]:
for name, parameters in model.named_parameters():
    if ('conv' in name) and ('bias' in name):
        print(name, parameters)
        print('\n')

conv1.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)


conv2.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)


